In [ ]:
from matplotlib import font_manager, rc  # 한글사용 matplotlib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import csv
import codecs
import networkx as nx
import numpy as np
import json
import datetime
import tensorflow as tf
import jpype
from konlpy.tag import Okt
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame as df
InteractiveShell.ast_node_interactivity = 'all'
%matplotlib inline
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import pandas as pd
import numpy as np # linear algebra
from numpy import newaxis
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf 

from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, make_scorer, confusion_matrix, accuracy_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, PolynomialFeatures
import sklearn.preprocessing as pp
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn.model_selection as ms
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import seaborn as sns

import matplotlib
matplotlib.rcParams['font.family'].insert(0, 'Malgun Gothic')

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input,Dense,GRU, Embedding 
from tensorflow.python.keras.optimizers import RMSprop,Adam
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.python.keras.layers import LSTM, Dropout, Dense, Activation

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

print ('module import completed')

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from keras.optimizers import adam

def get_eval(y_test, y_pred) :
    mae_1 = mae(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    rmse=np.sqrt(mse(y_test, y_pred))
    print('mae :',mae_1 )
    print('rmse :', rmse)


import matplotlib as mpl
import matplotlib.pyplot as pyplot
mpl.rcParams['font.family'].insert(0, 'Malgun Gothic')
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import pandas as pd
import pandas as pd
import lightgbm as lgb

# 훈련데이터 처리

In [ ]:
df_afsnt = pd.read_csv('bigcontest/2019빅콘테스트_퓨처스리그2/AFSNT.csv',engine='python',encoding='cp949')
df_afsnt["STT"] = pd.to_datetime(df_afsnt["STT"], format = "%H:%M").dt.time
df_afsnt["ATT"] = pd.to_datetime(df_afsnt["ATT"], format = "%H:%M").dt.time

df_afsnt = df_afsnt.rename(columns={ 
    "SDT_YY": "year", "SDT_MM":"month", "SDT_DD":"day",  "SDT_DY":"wday", 
    "ARP":"origin", "ODP":"dest", 
    "FLO":"airline", "FLT":"flight" ,"REG":"tailnum", 
    "AOD":"is_arrive", 
    "IRR":"is_regular", 
    "STT":"sched_time", "ATT":"real_time", 
    "DLY":"is_delay", "DRR":"cause_delay", 
    "CNL":"is_cancel", "CNR":"cause_cancel"
})
df_afsnt["date"] = pd.to_datetime(df_afsnt[["year", "month", "day"]])
df_afsnt["sched_datetime"] = pd.to_datetime(df_afsnt['date'].astype(str) + " " + df_afsnt["sched_time"].astype(str))
df_afsnt["real_datetime"] = pd.to_datetime(df_afsnt['date'].astype(str) + " " + df_afsnt["real_time"].astype(str))

# delay 변수 생성
temp = df_afsnt["real_datetime"] - df_afsnt["sched_datetime"] 
temp = pd.DataFrame(temp.dt.components)
df_afsnt["delay"] = temp['minutes'] + 60 * temp['hours'] + 1440 * temp['days']
del temp

# 공항 테이블
df_afsnt['origin'] = df_afsnt['origin'].replace({
    'ARP1': '김포',
    'ARP2': '김해',
    'ARP3': '제주',
    'ARP4': '대구',
    'ARP5': '울산',
    'ARP6': '청주',
    'ARP7': '무안',
    'ARP8': '광주',
    'ARP9': '여수',
    'ARP10': '양양',
    'ARP11': '포항',
    'ARP12': '사천',
    'ARP13': '군산',
    'ARP14': '원주',
    'ARP15': '인천',    
})
df_afsnt['dest'] = df_afsnt['dest'].replace({
    'ARP1': '김포',
    'ARP2': '김해',
    'ARP3': '제주',
    'ARP4': '대구',
    'ARP5': '울산',
    'ARP6': '청주',
    'ARP7': '무안',
    'ARP8': '광주',
    'ARP9': '여수',
    'ARP10': '양양',
    'ARP11': '포항',
    'ARP12': '사천',
    'ARP13': '군산',
    'ARP14': '원주',
    'ARP15': '인천',    
})

# 항공사 테이블 
df_afsnt['airline'] = df_afsnt['airline'].replace({
    'A': '아시아나', 
    'I': '진에어',
    'J': '대한항공', 
    'F': '이스타',
    'H': '제주항공', 
    'L': '티웨이',
    'B': '에어부산', 
    'M': '코리아익스프레스에어'
})

# 결항 데이터 제거
# tailnum 널값 제거 4개 정도임 결항 제거하면
# ARP1 -> ARP1 으로 가는 이상 데이터 제거

df_afsnt = df_afsnt[df_afsnt['is_cancel']=='N']
df_afsnt = df_afsnt[~(df_afsnt['tailnum'].isnull())]
df_afsnt = df_afsnt[~((df_afsnt['origin']=='김포') & (df_afsnt['dest']=='김포'))]
df_afsnt = df_afsnt[(df_afsnt["airline"] != "C") & (df_afsnt["airline"] != "D") & (df_afsnt["airline"] != "K") & (df_afsnt["airline"] != "G") & \
         (df_afsnt["airline"] != "E")]


df_afsnt.index = list(range(len(df_afsnt)))

# 시간 범주화 함수 정의
def time_dicrete(stt):
    time_dic = []
    for i in range(len(stt)):
        st_base = str(stt[i])
        st = int(st_base.split(':')[0])*60 + int(st_base.split(':')[1])
        if st >= 300 and st < 480:
            time_dic.append('5시-8시')
        elif st >= 480 and st < 660:
            time_dic.append('8시-11시')
        elif st >= 660 and st < 840:
            time_dic.append('11시-14시')
        elif st >= 840 and st < 1020:
            time_dic.append('14시-17시')
        elif st >= 1020 and st < 1200:
            time_dic.append('17시-20시')
        elif st >= 1200 and st < 1380:
            time_dic.append('20시-23시')
        elif st >=1380 or st<300:
            time_dic.append('23시-3시')
    return time_dic  

stt = list(df_afsnt['sched_time'])

a = time_dicrete(stt)
df_afsnt['time_discrete'] = a

# 혼잡도 변수 생성
# +- 15/30분 기준으로 운행하는 비행기의 수

In [ ]:
# +- 30분 기준 혼잡도 변수 생성
# 오래걸림

# 출도착 구분

df_afsnt_d = df_afsnt[df_afsnt['is_arrive']=='D']
df_afsnt_a = df_afsnt[df_afsnt['is_arrive']=='A']

# 출발기준 혼잡도

df_afsnt_d = df_afsnt_d.sort_values(['origin','date','sched_time'])
df_afsnt_d.index = list(range(len(df_afsnt_d)))

base_time = datetime.timedelta(minutes=30)
congestion_d = []

for i in range(len(df_afsnt_d)):
    
    airport = df_afsnt_d['origin'][i]
    sched_time = df_afsnt_d['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time + base_time
    congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(df_afsnt_d)-30) else len(df_afsnt_d) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airport == df_afsnt_d['origin'][j]) and (lower_time <= df_afsnt_d['sched_datetime'][j]) and (df_afsnt_d['sched_datetime'][j] <= upper_time):
            congestion_cnt +=1
    congestion_d.append(congestion_cnt)
    
df_afsnt_d['congestion'] = congestion_d

# 도착 기준 혼잡도

df_afsnt_a = df_afsnt_a.sort_values(['dest','date','sched_time'])
df_afsnt_a.index = list(range(len(df_afsnt_a)))

base_time = datetime.timedelta(minutes=30)
congestion_a = []

for i in range(len(df_afsnt_a)):
    
    airport = df_afsnt_a['dest'][i]
    sched_time = df_afsnt_a['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time + base_time
    congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(df_afsnt_a)-30) else len(df_afsnt_a) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airport == df_afsnt_a['dest'][j]) and (lower_time <= df_afsnt_a['sched_datetime'][j]) and (df_afsnt_a['sched_datetime'][j] <= upper_time):
            congestion_cnt +=1
    congestion_a.append(congestion_cnt)
    
df_afsnt_a['congestion'] = congestion_a

# +- 15분 간격으로 혼잡도 변수 생성

# 출발기준 혼잡도

df_afsnt_d = df_afsnt_d.sort_values(['origin','date','sched_time'])
df_afsnt_d.index = list(range(len(df_afsnt_d)))

base_time = datetime.timedelta(minutes=15)
congestion_d = []

for i in range(len(df_afsnt_d)):
    
    airport = df_afsnt_d['origin'][i]
    sched_time = df_afsnt_d['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time + base_time
    congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(df_afsnt_d)-30) else len(df_afsnt_d) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airport == df_afsnt_d['origin'][j]) and (lower_time <= df_afsnt_d['sched_datetime'][j]) and (df_afsnt_d['sched_datetime'][j] <= upper_time):
            congestion_cnt +=1
    congestion_d.append(congestion_cnt)
    
df_afsnt_d['congestion_15'] = congestion_d

# 도착 기준 혼잡도

df_afsnt_a = df_afsnt_a.sort_values(['dest','date','sched_time'])
df_afsnt_a.index = list(range(len(df_afsnt_a)))

base_time = datetime.timedelta(minutes=15)
congestion_a = []

for i in range(len(df_afsnt_a)):
    
    airport = df_afsnt_a['dest'][i]
    sched_time = df_afsnt_a['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time + base_time
    congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(df_afsnt_a)-30) else len(df_afsnt_a) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airport == df_afsnt_a['dest'][j]) and (lower_time <= df_afsnt_a['sched_datetime'][j]) and (df_afsnt_a['sched_datetime'][j] <= upper_time):
            congestion_cnt +=1
    congestion_a.append(congestion_cnt)
    
df_afsnt_a['congestion_15'] = congestion_a

# 링크 및 air_sequence 변수 생성
# 링크는 동일 등록기호의 연결성을 파악하기 위한 번호(시간 기준으로 번호)
# air_sequence는 공항에서 시간순으로 정렬한 번호

In [ ]:
# 링크데이터를 위한 정렬

df_afsnt_sorted_d = df_afsnt_d.sort_values(['date','tailnum','sched_datetime'])
df_afsnt_sorted_a = df_afsnt_a.sort_values(['date','tailnum','sched_datetime'])

df_afsnt_sorted_d.index = list(range(len(df_afsnt_sorted_d)))
link_ref_d = df_afsnt_sorted_d.pivot_table(index = ['date','tailnum'],aggfunc='count',values='is_arrive' )
link_ref_list_d = list(link_ref_d['is_arrive'].values)

#linkdata - departure

link_d = []

for i in link_ref_list_d:
    link_range = list(range(i))
    link_d.append(link_range)
    
link_d = sum(link_d, [])
df_afsnt_sorted_d['link'] = link_d


df_afsnt_sorted_a.index = list(range(len(df_afsnt_sorted_a)))
link_ref_a = df_afsnt_sorted_a.pivot_table(index = ['date','tailnum'],aggfunc='count',values='is_arrive' )
link_ref_list_a = list(link_ref_a['is_arrive'].values)

#linkdata - arrival

link_a = []

for i in link_ref_list_a:
    link_range = list(range(i))
    link_a.append(link_range)
    
link_a = sum(link_a, [])
df_afsnt_sorted_a['link'] = link_a

df_afsnt = pd.concat([df_afsnt_sorted_d, df_afsnt_sorted_a], sort = False)

df_afsnt_d = df_afsnt[df_afsnt['is_arrive']=='D']
df_afsnt_a = df_afsnt[df_afsnt['is_arrive']=='A']
df_afsnt_d = df_afsnt_d.sort_values(['date','origin','sched_time'])
df_afsnt_a = df_afsnt_a.sort_values(['date','dest','sched_time'])

df_afsnt_d.index = list(range(len(df_afsnt_d)))
df_afsnt_a.index = list(range(len(df_afsnt_a)))

air_sequence = []
x = 0

for i in range(len(df_afsnt_d)):
    if i == 0:
        air_sequence.append(0)
    elif (df_afsnt_d['date'][i] == df_afsnt_d['date'][i-1]) and (df_afsnt_d['origin'][i] == df_afsnt_d['origin'][i-1]):
            if df_afsnt_d['sched_datetime'][i] > df_afsnt_d['sched_datetime'][i-1]:
                x +=1
                air_sequence.append(x)
            elif df_afsnt_d['sched_datetime'][i] == df_afsnt_d['sched_datetime'][i-1]:
                air_sequence.append(x)
            elif df_afsnt_d['sched_datetime'][i] < df_afsnt_d['sched_datetime'][i-1]:
                x = 0
                air_sequence.append(x)
    
    else :
        x = 0
        air_sequence.append(x)

df_afsnt_d['air_sequence'] = air_sequence

air_sequence = []
x = 0

for i in range(len(df_afsnt_a)):
    if i == 0:
        air_sequence.append(0)
    elif (df_afsnt_a['date'][i] == df_afsnt_a['date'][i-1]) and (df_afsnt_a['dest'][i] == df_afsnt_a['dest'][i-1]):
            if df_afsnt_a['sched_datetime'][i] > df_afsnt_a['sched_datetime'][i-1]:
                x +=1
                air_sequence.append(x)
            elif df_afsnt_a['sched_datetime'][i] == df_afsnt_a['sched_datetime'][i-1]:
                air_sequence.append(x)
            elif df_afsnt_a['sched_datetime'][i] < df_afsnt_a['sched_datetime'][i-1]:
                x = 0
                air_sequence.append(x)
    
    else :
        x = 0
        air_sequence.append(x)

df_afsnt_a['air_sequence'] = air_sequence

df_afsnt = pd.concat([df_afsnt_d, df_afsnt_a], sort = False)

# 중심성 분석 결과 변수 생성

In [ ]:
# 링크테이블

# 출발 링크테이블

df_afsnt_d = df_afsnt[df_afsnt['is_arrive']=='D']

link_table_d = df_afsnt_d.pivot_table(index=['origin','dest','link'],aggfunc='count',values='is_arrive')
link_table_d = link_table_d.reset_index(['origin','dest','link'])
link_table_d.columns = ['origin','dest','link','count']

# 도착 링크

df_afsnt_a = df_afsnt[df_afsnt['is_arrive']=='A']

link_table_a = df_afsnt_a.pivot_table(index=['origin','dest','link'],aggfunc='count',values='is_arrive')
link_table_a = link_table_a.reset_index(['origin','dest','link'])
link_table_a.columns = ['origin','dest','link','count']

# 전체 링크

link_table =  df_afsnt.pivot_table(index=['origin','dest','link'],aggfunc='count',values='is_arrive')
link_table = link_table.reset_index(['origin','dest','link'])
link_table.columns = ['origin','dest','link','count']

# 테스트셋 테이블 생성
# link_table_d.to_csv('bigcontest/test_ref/link_table_d.csv', encoding = 'cp949', index = False)
# link_table_a.to_csv('bigcontest/test_ref/link_table_a.csv', encoding = 'cp949', index = False)
# link_table.to_csv('bigcontest/test_ref/link_table.csv', encoding = 'cp949', index = False)


# 중심성 분석 함수
# 가중치 고려

def get_degree_centrality(df,arp) :
    N = 15
    Z = df[df['origin']==arp]['count'].sum()
    D = Z/(N-1)
    return D


# 가중치 고려 x

def degree_centrality(df,arp):
    N =15
    a = df[df['origin']==arp]
    D = len((a['origin']+','+a['dest']).unique())/(N-1)
    return D

# 중심성 분석 

arp_list = list(link_table_d['origin'].unique())
center_link_d = pd.DataFrame(data=arp_list,columns=['origin'])
D_list = []

for i in arp_list:
    D = get_degree_centrality(link_table_d, i)
    D_list.append(D)

center_link_d['degree_center_weight'] = D_list

# 가중치 고려x

N =15
arp_list = list(link_table_d['origin'].unique())
center_link2_d= pd.DataFrame(data=arp_list,columns=['origin'])
D_list = []

for i in arp_list:
    D = degree_centrality(link_table_d, i)
    D_list.append(D)
    
center_link2_d['degree_center'] = D_list

center_link_d_total = pd.merge(center_link_d, center_link2_d)

# 원 데이터에 매핑

ref_dict = {}

for i in range(len(center_link_d_total)):
    ref_dict[center_link_d_total['origin'][i]] = center_link_d_total['degree_center_weight'][i]
    
ref_dict2 = {}
for i in range(len(center_link_d_total)):
    ref_dict2[center_link_d_total['origin'][i]] = center_link_d_total['degree_center'][i]

df_afsnt_d['degree_center_weight_origin'] = df_afsnt_d['origin'].map(ref_dict)
df_afsnt_d['degree_center_weight_dest'] = df_afsnt_d['dest'].map(ref_dict)
df_afsnt_d['degree_center_origin'] = df_afsnt_d['origin'].map(ref_dict2)
df_afsnt_d['degree_center_dest'] = df_afsnt_d['dest'].map(ref_dict2)

# 중심성 분석 

arp_list = list(link_table_a['origin'].unique())
center_link_a = pd.DataFrame(data=arp_list,columns=['origin'])
D_list = []

for i in arp_list:
    D = get_degree_centrality(link_table_a, i)
    D_list.append(D)

center_link_a['degree_center_weight'] = D_list

# 가중치 고려x

N =15
arp_list = list(link_table_a['origin'].unique())
center_link2_a= pd.DataFrame(data=arp_list,columns=['origin'])
D_list = []

for i in arp_list:
    D = degree_centrality(link_table_a, i)
    D_list.append(D)
    
center_link2_a['degree_center'] = D_list
center_link_a_total = pd.merge(center_link_a, center_link2_a)

# 원데이터에 매핑

ref_dict = {}

for i in range(len(center_link_a_total)):
    ref_dict[center_link_a_total['origin'][i]] = center_link_a_total['degree_center_weight'][i]
    
ref_dict2 = {}
for i in range(len(center_link_a_total)):
    ref_dict2[center_link_a_total['origin'][i]] = center_link_a_total['degree_center'][i]

df_afsnt_a['degree_center_weight_origin'] = df_afsnt_a['origin'].map(ref_dict)
df_afsnt_a['degree_center_weight_dest'] = df_afsnt_a['dest'].map(ref_dict)
df_afsnt_a['degree_center_origin'] = df_afsnt_a['origin'].map(ref_dict2)
df_afsnt_a['degree_center_dest'] = df_afsnt_a['dest'].map(ref_dict2)

df_afsnt = pd.concat([df_afsnt_d,df_afsnt_a], sort = False)

# 항공 흐름 혼잡도 변수 생성
# 같은 노선에 혼잡정도를 파악

In [ ]:
# 항공흐름 혼잡도 / 출도착 구분 
# 오래걸림

df_afsnt['origin_dest'] = df_afsnt['origin']+','+df_afsnt['dest']
# 15분 전 항공흐름

# 출도착 구분

df_afsnt_d = df_afsnt[df_afsnt['is_arrive']=='D']
df_afsnt_a = df_afsnt[df_afsnt['is_arrive']=='A']

df_afsnt_d = df_afsnt_d.sort_values(['date','origin_dest','sched_time'])
df_afsnt_a = df_afsnt_a.sort_values(['date','origin_dest','sched_time'])

df_afsnt_d.index = list(range(len(df_afsnt_d)))

base_time = datetime.timedelta(minutes=15)
air_congestion_d = []

for i in range(len(df_afsnt_d)):
    
    airway = df_afsnt_d['origin_dest'][i]
    sched_time = df_afsnt_d['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time 
    air_congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(df_afsnt_d)-30) else len(df_afsnt_d) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airway == df_afsnt_d['origin_dest'][j]) and (lower_time <= df_afsnt_d['sched_datetime'][j]) and (df_afsnt_d['sched_datetime'][j] <= upper_time):
            air_congestion_cnt +=1
    air_congestion_d.append(air_congestion_cnt)
    
df_afsnt_d['air_congestion_15'] = air_congestion_d


df_afsnt_a.index = list(range(len(df_afsnt_a)))

base_time = datetime.timedelta(minutes=15)
air_congestion_a = []

for i in range(len(df_afsnt_a)):
    
    airway = df_afsnt_a['origin_dest'][i]
    sched_time = df_afsnt_a['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time 
    air_congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(df_afsnt_a)-30) else len(df_afsnt_a) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airway == df_afsnt_a['origin_dest'][j]) and (lower_time <= df_afsnt_a['sched_datetime'][j]) and (df_afsnt_a['sched_datetime'][j] <= upper_time):
            air_congestion_cnt +=1
    air_congestion_a.append(air_congestion_cnt)
    
df_afsnt_a['air_congestion_15'] = air_congestion_a

df_afsnt = pd.concat([df_afsnt_d, df_afsnt_a], sort= False)

# 항공흐름 혼잡도 / 출도착 구분 
# 30분 전 항공흐름

# 출도착 구분

df_afsnt_d = df_afsnt[df_afsnt['is_arrive']=='D']
df_afsnt_a = df_afsnt[df_afsnt['is_arrive']=='A']

df_afsnt_d = df_afsnt_d.sort_values(['date','origin_dest','sched_time'])
df_afsnt_a = df_afsnt_a.sort_values(['date','origin_dest','sched_time'])

df_afsnt_d.index = list(range(len(df_afsnt_d)))

base_time = datetime.timedelta(minutes=30)
air_congestion_d = []

for i in range(len(df_afsnt_d)):
    
    airway = df_afsnt_d['origin_dest'][i]
    sched_time = df_afsnt_d['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time 
    air_congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(df_afsnt_d)-30) else len(df_afsnt_d) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airway == df_afsnt_d['origin_dest'][j]) and (lower_time <= df_afsnt_d['sched_datetime'][j]) and (df_afsnt_d['sched_datetime'][j] <= upper_time):
            air_congestion_cnt +=1
    air_congestion_d.append(air_congestion_cnt)
    
df_afsnt_d['air_congestion_30'] = air_congestion_d


df_afsnt_a.index = list(range(len(df_afsnt_a)))

base_time = datetime.timedelta(minutes=30)
air_congestion_a = []

for i in range(len(df_afsnt_a)):
    
    airway = df_afsnt_a['origin_dest'][i]
    sched_time = df_afsnt_a['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time 
    air_congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(df_afsnt_a)-30) else len(df_afsnt_a) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airway == df_afsnt_a['origin_dest'][j]) and (lower_time <= df_afsnt_a['sched_datetime'][j]) and (df_afsnt_a['sched_datetime'][j] <= upper_time):
            air_congestion_cnt +=1
    air_congestion_a.append(air_congestion_cnt)
    
df_afsnt_a['air_congestion_30'] = air_congestion_a

df_afsnt = pd.concat([df_afsnt_d, df_afsnt_a], sort= False)

# 항공사별 비행기의 평균 연결정도 변수 파악

In [ ]:
# 항공사의 일별 연결정도 변수

df_afsnt_d = df_afsnt[df_afsnt['is_arrive']=='D']
df_afsnt_a = df_afsnt[df_afsnt['is_arrive']=='A']

# 출도착 일별 평균 비행기 개수

d = df_afsnt_d.pivot_table(index = ['date', 'airline','tailnum'], aggfunc = 'count', values = 'is_arrive')
d = d.reset_index(['date','airline','tailnum'])
d2 = d.pivot_table(index = ['date','airline'], aggfunc = 'count', values = 'tailnum')
d2 = d2.reset_index(['date','airline'])
dd =d2.pivot_table(index = 'airline', aggfunc = 'mean', values = 'tailnum')
dd = dd.reset_index(['airline'])
dd.columns = ['airline','count']

a = df_afsnt_a.pivot_table(index = ['date', 'airline','tailnum'], aggfunc = 'count', values = 'is_arrive')
a = a.reset_index(['date','airline','tailnum'])
a2 = a.pivot_table(index = ['date','airline'], aggfunc = 'count', values = 'tailnum')
a2 = a2.reset_index(['date','airline'])
aa =a2.pivot_table(index = 'airline', aggfunc = 'mean', values = 'tailnum')
aa = aa.reset_index(['airline'])
aa.columns = ['airline','count']

# 테스트셋 테이블
# dd.to_csv('bigcontest/test_ref/dd.csv', index = False, encoding = 'cp949')
# aa.to_csv('bigcontest/test_ref/aa.csv', index = False, encoding = 'cp949')

x = df_afsnt_d.pivot_table(index = ['date','airline'],aggfunc = 'count', values = 'is_arrive')
x = x.reset_index(['date','airline'])
x.columns = ['date','airline','count']

y = df_afsnt_a.pivot_table(index = ['date','airline'],aggfunc = 'count', values = 'is_arrive')
y = y.reset_index(['date','airline'])
y.columns = ['date','airline','count']

link_count = []
for i in range(len(x)):
    a = x['count'][i]
    b = float(dd[dd['airline']==x['airline'][i]]['count'].values)
    cnt = a/b
    link_count.append(cnt)
x['link_count'] = link_count

link_count = []
for i in range(len(y)):
    a = y['count'][i]
    b = float(aa[aa['airline']==y['airline'][i]]['count'].values)
    cnt = a/b
    link_count.append(cnt)
y['link_count'] = link_count

x = x.drop(columns = 'count')
y = y.drop(columns = 'count')

df_afsnt_d = pd.merge(df_afsnt_d,x, on = ['date','airline'])
df_afsnt_a = pd.merge(df_afsnt_a, y, on = ['date','airline'])

df_afsnt = pd.concat([df_afsnt_d, df_afsnt_a], sort = False)

# 항공사별 시간순 변수 생성

In [ ]:
# 항공사별 sequence 생성

df_afsnt_d = df_afsnt[df_afsnt['is_arrive']=='D']
df_afsnt_a = df_afsnt[df_afsnt['is_arrive']=='A']

df_afsnt_d = df_afsnt_d.sort_values(['date','airline','sched_time'])
df_afsnt_a = df_afsnt_a.sort_values(['date','airline','sched_time'])

df_afsnt_d.index = list(range(len(df_afsnt_d)))
df_afsnt_a.index = list(range(len(df_afsnt_a)))

# 출발

air_link = []
x = 0

for i in range(len(df_afsnt_d)):
    if i == 0:
        air_link.append(0)
    
    elif df_afsnt_d['sched_datetime'][i] > df_afsnt_d['sched_datetime'][i-1]:
        if df_afsnt_d['day'][i] == df_afsnt_d['day'][i-1]:
            x += 1
            air_link.append(x)
        elif df_afsnt_d['day'][i] > df_afsnt_d['day'][i-1]:
            x = 0
            air_link.append(x)
        elif df_afsnt_d['day'][i] < df_afsnt_d['day'][i-1]:
            x = 0
            air_link.append(x)
        
    elif df_afsnt_d['sched_datetime'][i] == df_afsnt_d['sched_datetime'][i-1]:
        air_link.append(x)
        
    elif df_afsnt_d['sched_datetime'][i] < df_afsnt_d['sched_datetime'][i-1]:
        x = 0
        air_link.append(x)

df_afsnt_d['air_link'] = air_link

# 도착

air_link = []
x = 0

for i in range(len(df_afsnt_a)):
    if i == 0:
        air_link.append(0)
    
    elif df_afsnt_a['sched_datetime'][i] > df_afsnt_a['sched_datetime'][i-1]:
        if df_afsnt_a['day'][i] == df_afsnt_a['day'][i-1]:
            x += 1
            air_link.append(x)
        elif df_afsnt_a['day'][i] > df_afsnt_a['day'][i-1]:
            x = 0
            air_link.append(x)
        elif df_afsnt_a['day'][i] < df_afsnt_a['day'][i-1]:
            x = 0
            air_link.append(x)
        
    elif df_afsnt_a['sched_datetime'][i] == df_afsnt_a['sched_datetime'][i-1]:
        air_link.append(x)
        
    elif df_afsnt_a['sched_datetime'][i] < df_afsnt_a['sched_datetime'][i-1]:
        x = 0
        air_link.append(x)
        

df_afsnt_a['air_link'] = air_link


# 공항별 항공사별 sequence 생성

df_afsnt_d = df_afsnt_d.sort_values(['date','origin','airline','sched_time'])
df_afsnt_a = df_afsnt_a.sort_values(['date','dest','airline','sched_time'])

df_afsnt_d.index = list(range(len(df_afsnt_d)))
df_afsnt_a.index = list(range(len(df_afsnt_a)))

air_place_link = []
x = 0

for i in range(len(df_afsnt_d)):
    if i == 0:
        air_place_link.append(0)
    
    elif (df_afsnt_d['origin'][i] == df_afsnt_d['origin'][i-1]) and (df_afsnt_d['date'][i] == df_afsnt_d['date'][i-1]) and (df_afsnt_d['airline'][i] == df_afsnt_d['airline'][i-1]):
        if df_afsnt_d['sched_datetime'][i] > df_afsnt_d['sched_datetime'][i-1]:
            x += 1
            air_place_link.append(x)
        elif df_afsnt_d['sched_datetime'][i] == df_afsnt_d['sched_datetime'][i-1]:
            air_place_link.append(x)
        elif df_afsnt_d['sched_datetime'][i] < df_afsnt_d['sched_datetime'][i-1]:
            x = 0
            air__placelink.append(x)
        
    else :
        x = 0
        air_place_link.append(x)

df_afsnt_d['air_place_link'] = air_place_link

air_place_link = []
x = 0

for i in range(len(df_afsnt_a)):
    if i == 0:
        air_place_link.append(0)
    
    elif (df_afsnt_a['dest'][i] == df_afsnt_a['dest'][i-1]) and (df_afsnt_a['date'][i] == df_afsnt_a['date'][i-1]) and (df_afsnt_a['airline'][i] == df_afsnt_a['airline'][i-1]):
        if df_afsnt_a['sched_datetime'][i] > df_afsnt_a['sched_datetime'][i-1]:
            x += 1
            air_place_link.append(x)
        elif df_afsnt_a['sched_datetime'][i] == df_afsnt_a['sched_datetime'][i-1]:
            air_place_link.append(x)
        elif df_afsnt_a['sched_datetime'][i] < df_afsnt_a['sched_datetime'][i-1]:
            x = 0
            air__placelink.append(x)
        
    else :
        x = 0
        air_place_link.append(x)

df_afsnt_a['air_place_link'] = air_place_link

df_afsnt = pd.concat([df_afsnt_d, df_afsnt_a], sort = False)

# 거리, 공항 관련 데이터(부지면적, 활주로 등), 거리중심성 변수 생성

In [ ]:
# 거리 매핑 및 거리중심성 

distance = pd.read_csv('bigcontest/airport_distance.csv', engine = 'python', encoding = 'cp949')
distance['origin_dest'] = distance['origin']+','+distance['dest']
df_afsnt['origin_dest'] = df_afsnt['origin']+','+df_afsnt['dest']

ref_dict = {}

for i in range(len(distance)):
    ref_dict[distance['origin_dest'][i]] = distance['distance_km'][i]

df_afsnt['distance_km'] = df_afsnt['origin_dest'].map(ref_dict)

G = nx.from_pandas_edgelist(df_afsnt, source = "origin", target= "dest")
closeness_centrality = nx.closeness_centrality(G)
node, BC = zip(*closeness_centrality.items())
center = pd.DataFrame({'origin':node, 'distance_center':BC}).sort_values(by='distance_center',ascending=False)

# 테스트셋 테이블 생성
# center.to_csv('bigcontest/test_ref/center.csv', index = False, encoding = 'cp949')

# 매핑

ref_dict = {}

for i in range(len(center)):
    ref_dict[center['origin'][i]] = center['distance_center'][i]

df_afsnt['distance_center'] = df_afsnt['origin'].map(ref_dict)

# 거리중심성 분석
# 어떤 테이블이 있을건데 이것은 출발 - 도착 - 거리를 보여주는 데이터프레임 = df_distance 

origin_unique = (df_afsnt['origin']).unique()

def get_distance_centrality(origin):
    N = 15
    df_distance = distance[distance['origin']==origin]
    df_distance.index = list(range(len(df_distance)))
    distance_sum = 0
    for i in range(len(df_distance)):
        distance_sum += df_distance['distance_km'][i]
            
    distance_sum_inverse = (distance_sum)**(-1)
    distance_center = (N-1)*distance_sum_inverse
    return distance_center 

# 중심성 분석 

origin_unique = (df_afsnt['origin']).unique()
center_distance = pd.DataFrame(data=origin_unique,columns=['origin'])
D_list = []

for i in origin_unique:
    D = get_distance_centrality(i)
    D_list.append(D)

center_distance['distance_center'] = D_list

ref_dict = {}

for i in range(len(center_distance)):
    ref_dict[center_distance['origin'][i]] = center_distance['distance_center'][i]

df_afsnt['distance_center_weight'] = df_afsnt['origin'].map(ref_dict)

# 공항정보 매핑

info = pd.read_csv('bigcontest/공항활주로_부지면적.csv', engine='python',encoding='cp949')
df_afsnt = pd.merge(df_afsnt, info, on=['origin','year'])
df_afsnt = df_afsnt.drop(columns = ['origin_dest'])

# 하루당 동일 비행기의 운행횟수 변수

In [ ]:
# 하루당 해당 비행기 운행횟수

df_afsnt_d = df_afsnt[df_afsnt['is_arrive']=='D']
df_afsnt_a = df_afsnt[df_afsnt['is_arrive']=='A']

d_table = df_afsnt_d.pivot_table(index=['date','tailnum'],aggfunc='count',values='is_arrive')
d_table = d_table.reset_index(['date','tailnum'])
d_table.columns = ['date','tailnum','cnt_per_day']

a_table = df_afsnt_a.pivot_table(index=['date','tailnum'],aggfunc='count',values='is_arrive')
a_table = a_table.reset_index(['date','tailnum'])
a_table.columns = ['date','tailnum','cnt_per_day']

df_afsnt_d2 = pd.merge(df_afsnt_d, d_table, on = ['date','tailnum'])
df_afsnt_a2 = pd.merge(df_afsnt_a, a_table, on = ['date','tailnum'])

df_afsnt = pd.concat([df_afsnt_d2,df_afsnt_a2], sort = False)

# 공항 관련 데이터, 시간 데이터 추가

In [ ]:
df_afsnt['is_flight_T'] = np.where(df_afsnt['flight'].str.endswith("T"), 1, 0)
df_afsnt['is_flight_M'] = np.where(df_afsnt['flight'].str.endswith("M"), 1, 0)
df_afsnt['is_flight_F'] = np.where(df_afsnt['flight'].str.endswith("F"), 1, 0)

import datetime as dt

df_afsnt['sched_datetime'] = pd.to_datetime(df_afsnt['sched_datetime'])

# 월
df_afsnt['month_sin'] = np.sin(2*df_afsnt['sched_datetime'].dt.month*np.pi / 12 )
df_afsnt['month_cos'] = np.cos(2*df_afsnt['sched_datetime'].dt.month*np.pi / 12 )

# 요일
df_afsnt['sched_datetime'] = pd.to_datetime(df_afsnt['sched_datetime'])
df_afsnt['is_weekend'] = np.where((df_afsnt['sched_datetime'].dt.weekday == 6) | (df_afsnt['sched_datetime'].dt.weekday == 5), True, False)

# 시간
df_afsnt['time_sin'] = 60 * np.sin(2*np.pi*df_afsnt['sched_datetime'].dt.hour / 24) + \
                        np.sin(2*df_afsnt['sched_datetime'].dt.minute*np.pi / 60)
df_afsnt['time_cos'] = 60 * np.cos(2*df_afsnt['sched_datetime'].dt.hour*np.pi / 24) + \
                        np.cos(2*np.pi*df_afsnt['sched_datetime'].dt.minute / 60)




# 해발 고도
df_altitude = pd.DataFrame(pd.Series({
    '김포': 18,
    '김해': 4,
    '제주': 36,
    '대구': 35.4,
    '울산': 12.9,
    '청주': 58,
    '양양': 73.5,
    '무안': 23.7,
    '광주': 12.8,
    '여수': 20,
    '사천': 7.6,
    '포항': 21.3,
    '군산': 8.8,
    '원주': 100,
    '인천': 7
})).reset_index().rename(columns = {'index': 'airport', 0: 'alt'})

df_afsnt = pd.merge(df_afsnt, df_altitude, how = "left", left_on = 'origin', right_on = 'airport', suffixes = ('_origin', '_dest'))
df_afsnt = pd.merge(df_afsnt, df_altitude, how = "left", left_on = 'dest', right_on = 'airport', suffixes = ('_origin', '_dest'))
df_afsnt.drop(['airport_origin', 'airport_dest'], axis = 1, inplace = True)


# is_변수 True False로 변환
df_afsnt["is_arrive"] = np.where(df_afsnt["is_arrive"] == "A", True, False)
df_afsnt["is_regular"] = np.where(df_afsnt["is_regular"] == "Y", True, False)
df_afsnt["is_delay"] = np.where(df_afsnt["is_delay"] == "Y", True, False)

# 추가 변수들
df_afsnt['origin_dest'] = df_afsnt['origin'] + '_' + df_afsnt['dest']



# 황금연휴 추가

df_afsnt['is_goldentime'] = df_afsnt['date'].isin(["2017-01-26", "2017-01-27", "2017-01-28", "2017-01-29", "2017-01-30", "2017-01-31", "2017-05-05", "2017-05-06", "2017-05-07", "2017-08-15",
 "2017-09-29", "2017-09-30", "2017-10-01", "2017-10-02", "2017-10-03", "2017-10-04", "2017-10-05", "2017-10-06", "2017-10-07", "2017-10-08", "2017-10-09", "2017-10-10", 
                                                   "2017-12-25", "2018-01-01", "2018-02-14", "2018-02-15", "2018-02-16", "2018-02-17", "2018-02-18", "2018-02-19", "2018-03-01",
 "2018-05-05", "2018-05-07", "2018-05-22",
 "2018-06-06", "2018-06-13", "2018-08-15", "2018-09-21", "2018-09-22", "2018-09-23", "2018-09-24", "2018-09-25", "2018-09-26", "2018-09-27", 
 "2018-10-03", "2018-10-09", "2018-12-25", 
 "2019-01-01", "2019-02-01", "2019-02-02", "2019-02-03", "2019-02-04", "2019-02-05", "2019-02-06", "2019-02-07",
 "2019-03-01", "2019-05-04", "2019-05-05", "2019-05-06"])

# 전비행기와의 시간차 및 거리대비 시간차 변수 생성

In [ ]:
# 연결 비행기에 있어서 전 비행기와의 시간차이 및 거리를 고려한 시간차이 


df_afsnt_d = df_afsnt[df_afsnt['is_arrive']==False]
df_afsnt_a = df_afsnt[df_afsnt['is_arrive']==True]

# 출발인경우

df_afsnt_d_sorted = df_afsnt_d.sort_values(['date','tailnum','sched_time'])
df_afsnt_d_sorted.index = list(range(len(df_afsnt_d_sorted)))

time_diff = []
time_diff_distance = []

for i in range(len(df_afsnt_d_sorted)):
    if df_afsnt_d_sorted['link'][i] == 0 :
        time_diff.append(0)
        time_diff_distance.append(0)
        
    elif df_afsnt_d_sorted['tailnum'][i] != df_afsnt_d_sorted['tailnum'][i-1]:
        time_diff.append(0)
        time_diff_distance.append(0)
        
    else :
        time_diff_sec = (df_afsnt_d_sorted['sched_datetime'][i] - df_afsnt_d_sorted['sched_datetime'][i-1]).seconds
        time_diff_min = int(time_diff_sec/60)
        time_diff.append(time_diff_min)
        
        time_diff_dis = time_diff_min/df_afsnt_d_sorted['distance_km'][i-1]
        time_diff_distance.append(time_diff_dis)
        
df_afsnt_d_sorted['time_diff'] = time_diff
df_afsnt_d_sorted['time_diff_distance'] = time_diff_distance

# 도착인경우

df_afsnt_a_sorted = df_afsnt_a.sort_values(['date','tailnum','sched_time'])
df_afsnt_a_sorted.index = list(range(len(df_afsnt_a_sorted)))

time_diff = []
time_diff_distance = []

for i in range(len(df_afsnt_a_sorted)):
    if df_afsnt_a_sorted['link'][i] == 0 :
        time_diff.append(0)
        time_diff_distance.append(0)
        
    elif df_afsnt_a_sorted['tailnum'][i] != df_afsnt_a_sorted['tailnum'][i-1]:
        time_diff.append(0)
        time_diff_distance.append(0)
        
    else :
        time_diff_sec = (df_afsnt_a_sorted['sched_datetime'][i] - df_afsnt_a_sorted['sched_datetime'][i-1]).seconds
        time_diff_min = int(time_diff_sec/60)
        time_diff.append(time_diff_min)
        
        time_diff_dis = time_diff_min/df_afsnt_a_sorted['distance_km'][i-1]
        time_diff_distance.append(time_diff_dis)
        
df_afsnt_a_sorted['time_diff'] = time_diff
df_afsnt_a_sorted['time_diff_distance'] = time_diff_distance
df_afsnt = pd.concat([df_afsnt_d_sorted, df_afsnt_a_sorted], sort = False)

# 항공편 지연비율 매핑 변수

In [ ]:
df_afsnt_d = df_afsnt[df_afsnt['is_arrive']==False]
df_afsnt_a = df_afsnt[df_afsnt['is_arrive']==True]

d_x = df_afsnt_d[df_afsnt_d['is_delay']==True].pivot_table(index = 'flight', aggfunc = 'count', values ='is_arrive')
d_y = df_afsnt_d.pivot_table(index = 'flight', aggfunc = 'count', values ='is_arrive')
a_x = df_afsnt_a[df_afsnt_a['is_delay']==True].pivot_table(index = 'flight', aggfunc = 'count', values ='is_arrive')
a_y = df_afsnt_a.pivot_table(index = 'flight', aggfunc = 'count', values ='is_arrive')

d = d_x/d_y
d = d.reset_index('flight')
d.columns  = ['flight','dly_por']
a = a_x/a_y
a = a.reset_index('flight')
a.columns = ['flight','dly_por']

d.fillna(d['dly_por'].mean(),inplace=True)
a.fillna(a['dly_por'].mean(),inplace=True)

df_afsnt_d = pd.merge(df_afsnt_d, d, on = 'flight')
df_afsnt_a = pd.merge(df_afsnt_a, a, on = 'flight')
df_afsnt = pd.concat([df_afsnt_d, df_afsnt_a], sort=False)

# 테스트셋에서의 활용을 위한 테이블
# d.to_csv('bigcontest/test_ref/flight_dly_d.csv', index = False, encoding = 'cp949')
# a.to_csv('bigcontest/test_ref/flight_dly_a.csv', index = False, encoding = 'cp949')

df_afsnt.to_csv('bigcontest/df_afsnt_total_ver10.csv', index = False, encoding = 'cp949')

# 테스트셋 처리

In [ ]:
test = pd.read_csv('bigcontest/2019빅콘테스트_퓨처스리그2/AFSNT_DLY.csv',engine='python',encoding='cp949')
test["STT"] = pd.to_datetime(test["STT"], format = "%H:%M").dt.time

test = test.rename(columns={ 
    "SDT_YY": "year", "SDT_MM":"month", "SDT_DD":"day",  "SDT_DY":"wday", 
    "ARP":"origin", "ODP":"dest", 
    "FLO":"airline", "FLT":"flight" , 
    "AOD":"is_arrive",  
    "STT":"sched_time",
    "DLY":"is_delay", 
})
test["date"] = pd.to_datetime(test[["year", "month", "day"]])
test["sched_datetime"] = pd.to_datetime(test['date'].astype(str) + " " + test["sched_time"].astype(str))


# 공항 테이블
test['origin'] = test['origin'].replace({
    'ARP1': '김포',
    'ARP2': '김해',
    'ARP3': '제주',
    'ARP4': '대구',
    'ARP5': '울산',
    'ARP6': '청주',
    'ARP7': '무안',
    'ARP8': '광주',
    'ARP9': '여수',
    'ARP10': '양양',
    'ARP11': '포항',
    'ARP12': '사천',
    'ARP13': '군산',
    'ARP14': '원주',
    'ARP15': '인천',    
})
test['dest'] = test['dest'].replace({
    'ARP1': '김포',
    'ARP2': '김해',
    'ARP3': '제주',
    'ARP4': '대구',
    'ARP5': '울산',
    'ARP6': '청주',
    'ARP7': '무안',
    'ARP8': '광주',
    'ARP9': '여수',
    'ARP10': '양양',
    'ARP11': '포항',
    'ARP12': '사천',
    'ARP13': '군산',
    'ARP14': '원주',
    'ARP15': '인천',    
})

# 항공사 테이블 
test['airline'] = test['airline'].replace({
    'A': '아시아나', 
    'I': '진에어',
    'J': '대한항공', 
    'F': '이스타',
    'H': '제주항공', 
    'L': '티웨이',
    'B': '에어부산', 
    'M': '코리아익스프레스에어'
})


test.index = list(range(len(test)))

# 시간 범주화 함수 정의
def time_dicrete(stt):
    time_dic = []
    for i in range(len(stt)):
        st_base = str(stt[i])
        st = int(st_base.split(':')[0])*60 + int(st_base.split(':')[1])
        if st >= 300 and st < 480:
            time_dic.append('5시-8시')
        elif st >= 480 and st < 660:
            time_dic.append('8시-11시')
        elif st >= 660 and st < 840:
            time_dic.append('11시-14시')
        elif st >= 840 and st < 1020:
            time_dic.append('14시-17시')
        elif st >= 1020 and st < 1200:
            time_dic.append('17시-20시')
        elif st >= 1200 and st < 1380:
            time_dic.append('20시-23시')
        elif st >=1380 or st<300:
            time_dic.append('23시-3시')
    return time_dic  

stt = list(test['sched_time'])

a = time_dicrete(stt)
test['time_discrete'] = a

# 등록기호 매핑
# 훈련데이터를 통해 매핑을 진행하였고 수작업으로 진행한 부분이 있어서 코드는 주석처리
# 파일을 통해 병합

In [ ]:
# 등록기호 매핑

# 해당 부분은 학습 데이터의 6월 17~23일까지의 일주일 데이터로 테스트셋의 등록기호를 매핑한 코드입니다.
# 작업한 후 몇 작업에 대해선 수작업으로 진행해야되기 때문에 코드화하지 못하였고
# 해당파일을 제출하여 읽어오는 방식입니다.

# df_afsnt = pd.read_csv('bigcontest/df_afsnt_total_ver10.csv', encoding = 'utf-8', engine = 'python')
# df_afsnt["sched_time"] = pd.to_datetime(df_afsnt["sched_time"], format = "%H:%M:%S").dt.time
# df_afsnt["real_time"] = pd.to_datetime(df_afsnt["real_time"], format = "%H:%M:%S").dt.time
# df_afsnt = df_afsnt[df_afsnt['date']> '2019-03-31']
# df_afsnt_d = df_afsnt[df_afsnt['is_arrive']== False ]
# df_afsnt_a = df_afsnt[df_afsnt['is_arrive']== True]
# df_afsnt_d = df_afsnt_d[(df_afsnt_d['month']==6) & (df_afsnt_d['day'] < 24) & (df_afsnt_d['day'] > 16)]
# df_afsnt_a = df_afsnt_a[(df_afsnt_a['month']==6) & (df_afsnt_a['day'] < 24) & (df_afsnt_a['day'] > 16)]
# test['tailnum'] = -999
# test['link'] = -999
# test_d = test[test['is_arrive']== 'D']
# test_a = test[test['is_arrive']== 'A']
# df_afsnt_d.index = list(range(len(df_afsnt_d)))
# df_afsnt_a.index = list(range(len(df_afsnt_a)))
# test_d.index = list(range(len(test_d)))
# test_a.index = list(range(len(test_a)))
# for i in range(len(test_d)):
#    wday = test_d['wday'][i]
#    origin = test_d['origin'][i]
#    dest = test_d['dest'][i]
#    sched_time = test_d['sched_time'][i]
#    flight = test_d['flight'][i]
#    for j in range(len(df_afsnt_d)):
#        if (wday == df_afsnt_d['wday'][j]) & (origin == df_afsnt_d['origin'][j]) & (dest == df_afsnt_d['dest'][j]) & (flight == df_afsnt_d['flight'][j])& (sched_time == df_afsnt_d['sched_time'][j]):
#            test_d['tailnum'][i] = df_afsnt_d['tailnum'][j]
#            test_d['link'][i] = df_afsnt_d['link'][j]
#for i in range(len(test_a)):
#    wday = test_a['wday'][i]
#    origin = test_a['origin'][i]
#    dest = test_a['dest'][i]
#    sched_time = test_a['sched_time'][i]
#    flight = test_a['flight'][i] 
#    for j in range(len(df_afsnt_a)):
#        if (wday == df_afsnt_a['wday'][j]) & (origin == df_afsnt_a['origin'][j]) & (dest == df_afsnt_a['dest'][j]) & (flight == df_afsnt_a['flight'][j])& (sched_time == df_afsnt_a['sched_time'][j]):
#            test_a['tailnum'][i] = df_afsnt_a['tailnum'][j]
#            test_a['link'][i] = df_afsnt_a['link'][j]

test_link_tailnum = pd.read_csv('bigcontest/test_ref/test_link_tailnum.csv', engine = 'python', encoding = 'cp949')
test_link_tailnum["sched_time"] = pd.to_datetime(test_link_tailnum["sched_time"], format = "%H:%M:%S").dt.time
test_link_tailnum = test_link_tailnum.drop(columns = ['time_discrete','sched_datetime','DLY_RATE','is_delay',
                                                     'date'])
test = pd.merge(test, test_link_tailnum, on = ['year','month','day','wday','origin','dest','airline',
                                               'flight','is_arrive','sched_time'])

# 혼잡도 변수 생성

In [ ]:
# 출도착 구분

test_d = test[test['is_arrive']=='D']
test_a = test[test['is_arrive']=='A']

# +- 30분 기준 혼잡도 변수 생성

# 출발기준 혼잡도

test_d = test_d.sort_values(['origin','date','sched_time'])
test_d.index = list(range(len(test_d)))

base_time = datetime.timedelta(minutes=30)
congestion_d = []

for i in range(len(test_d)):
    
    airport = test_d['origin'][i]
    sched_time = test_d['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time + base_time
    congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(test_d)-30) else len(test_d) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airport == test_d['origin'][j]) and (lower_time <= test_d['sched_datetime'][j]) and (test_d['sched_datetime'][j] <= upper_time):
            congestion_cnt +=1
    congestion_d.append(congestion_cnt)
    
test_d['congestion'] = congestion_d

# 도착 기준 혼잡도

test_a = test_a.sort_values(['dest','date','sched_time'])
test_a.index = list(range(len(test_a)))

base_time = datetime.timedelta(minutes=30)
congestion_a = []

for i in range(len(test_a)):
    
    airport = test_a['dest'][i]
    sched_time = test_a['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time + base_time
    congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(test_a)-30) else len(test_a) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airport == test_a['dest'][j]) and (lower_time <= test_a['sched_datetime'][j]) and (test_a['sched_datetime'][j] <= upper_time):
            congestion_cnt +=1
    congestion_a.append(congestion_cnt)
    
test_a['congestion'] = congestion_a

# +- 15분 간격으로 혼잡도 변수 생성

# 출발기준 혼잡도

test_d = test_d.sort_values(['origin','date','sched_time'])
test_d.index = list(range(len(test_d)))

base_time = datetime.timedelta(minutes=15)
congestion_d = []

for i in range(len(test_d)):
    
    airport = test_d['origin'][i]
    sched_time = test_d['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time + base_time
    congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(test_d)-30) else len(test_d) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airport == test_d['origin'][j]) and (lower_time <= test_d['sched_datetime'][j]) and (test_d['sched_datetime'][j] <= upper_time):
            congestion_cnt +=1
    congestion_d.append(congestion_cnt)
    
test_d['congestion_15'] = congestion_d

# 도착 기준 혼잡도

test_a = test_a.sort_values(['dest','date','sched_time'])
test_a.index = list(range(len(test_a)))

base_time = datetime.timedelta(minutes=15)
congestion_a = []

for i in range(len(test_a)):
    
    airport = test_a['dest'][i]
    sched_time = test_a['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time + base_time
    congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(test_a)-30) else len(test_a) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airport == test_a['dest'][j]) and (lower_time <= test_a['sched_datetime'][j]) and (test_a['sched_datetime'][j] <= upper_time):
            congestion_cnt +=1
    congestion_a.append(congestion_cnt)
    
test_a['congestion_15'] = congestion_a

test = pd.concat([test_d, test_a], sort = False)

# air_sequence 생성

In [ ]:
# 공항 sequence

test_d = test[test['is_arrive']=='D']
test_a = test[test['is_arrive']=='A']
test_d = test_d.sort_values(['date','origin','sched_time'])
test_a = test_a.sort_values(['date','dest','sched_time'])

test_d.index = list(range(len(test_d)))
test_a.index = list(range(len(test_a)))

air_sequence = []
x = 0

for i in range(len(test_d)):
    if i == 0:
        air_sequence.append(0)
    elif (test_d['date'][i] == test_d['date'][i-1]) and (test_d['origin'][i] == test_d['origin'][i-1]):
            if test_d['sched_datetime'][i] > test_d['sched_datetime'][i-1]:
                x +=1
                air_sequence.append(x)
            elif test_d['sched_datetime'][i] == test_d['sched_datetime'][i-1]:
                air_sequence.append(x)
            elif test_d['sched_datetime'][i] < test_d['sched_datetime'][i-1]:
                x = 0
                air_sequence.append(x)
    
    else :
        x = 0
        air_sequence.append(x)

test_d['air_sequence'] = air_sequence

air_sequence = []
x = 0

for i in range(len(test_a)):
    if i == 0:
        air_sequence.append(0)
    elif (test_a['date'][i] == test_a['date'][i-1]) and (test_a['dest'][i] == test_a['dest'][i-1]):
            if test_a['sched_datetime'][i] > test_a['sched_datetime'][i-1]:
                x +=1
                air_sequence.append(x)
            elif test_a['sched_datetime'][i] == test_a['sched_datetime'][i-1]:
                air_sequence.append(x)
            elif test_a['sched_datetime'][i] < test_a['sched_datetime'][i-1]:
                x = 0
                air_sequence.append(x)
    
    else :
        x = 0
        air_sequence.append(x)

test_a['air_sequence'] = air_sequence

test = pd.concat([test_d, test_a], sort = False)

# 중심성 변수

In [ ]:
# 링크테이블

# 출발 링크테이블

test_d = test[test['is_arrive']=='D']
link_table_d = pd.read_csv('bigcontest/test_ref/link_table_d.csv', encoding = 'cp949', engine = 'python')

# 도착 링크

test_a = test[test['is_arrive']=='A']
link_table_a = pd.read_csv('bigcontest/test_ref/link_table_a.csv', encoding = 'cp949', engine = 'python')


# 전체 링크

link_table = pd.read_csv('bigcontest/test_ref/link_table.csv', encoding = 'cp949', engine = 'python')



# 중심성 분석 함수
# 가중치 고려

def get_degree_centrality(df,arp) :
    N = 15
    Z = df[df['origin']==arp]['count'].sum()
    D = Z/(N-1)
    return D


# 가중치 고려 x

def degree_centrality(df,arp):
    N =15
    a = df[df['origin']==arp]
    D = len((a['origin']+','+a['dest']).unique())/(N-1)
    return D

# 중심성 분석 

arp_list = list(link_table_d['origin'].unique())
center_link_d = pd.DataFrame(data=arp_list,columns=['origin'])
D_list = []

for i in arp_list:
    D = get_degree_centrality(link_table_d, i)
    D_list.append(D)

center_link_d['degree_center_weight'] = D_list

# 가중치 고려x

N =15
arp_list = list(link_table_d['origin'].unique())
center_link2_d= pd.DataFrame(data=arp_list,columns=['origin'])
D_list = []

for i in arp_list:
    D = degree_centrality(link_table_d, i)
    D_list.append(D)
    
center_link2_d['degree_center'] = D_list

center_link_d_total = pd.merge(center_link_d, center_link2_d)

# 원 데이터에 매핑

ref_dict = {}

for i in range(len(center_link_d_total)):
    ref_dict[center_link_d_total['origin'][i]] = center_link_d_total['degree_center_weight'][i]
    
ref_dict2 = {}
for i in range(len(center_link_d_total)):
    ref_dict2[center_link_d_total['origin'][i]] = center_link_d_total['degree_center'][i]

test_d['degree_center_weight_origin'] = test_d['origin'].map(ref_dict)
test_d['degree_center_weight_dest'] = test_d['dest'].map(ref_dict)
test_d['degree_center_origin'] = test_d['origin'].map(ref_dict2)
test_d['degree_center_dest'] = test_d['dest'].map(ref_dict2)

# 중심성 분석 

arp_list = list(link_table_a['origin'].unique())
center_link_a = pd.DataFrame(data=arp_list,columns=['origin'])
D_list = []

for i in arp_list:
    D = get_degree_centrality(link_table_a, i)
    D_list.append(D)

center_link_a['degree_center_weight'] = D_list

# 가중치 고려x

N =15
arp_list = list(link_table_a['origin'].unique())
center_link2_a= pd.DataFrame(data=arp_list,columns=['origin'])
D_list = []

for i in arp_list:
    D = degree_centrality(link_table_a, i)
    D_list.append(D)
    
center_link2_a['degree_center'] = D_list
center_link_a_total = pd.merge(center_link_a, center_link2_a)

# 원데이터에 매핑

ref_dict = {}

for i in range(len(center_link_a_total)):
    ref_dict[center_link_a_total['origin'][i]] = center_link_a_total['degree_center_weight'][i]
    
ref_dict2 = {}
for i in range(len(center_link_a_total)):
    ref_dict2[center_link_a_total['origin'][i]] = center_link_a_total['degree_center'][i]

test_a['degree_center_weight_origin'] = test_a['origin'].map(ref_dict)
test_a['degree_center_weight_dest'] = test_a['dest'].map(ref_dict)
test_a['degree_center_origin'] = test_a['origin'].map(ref_dict2)
test_a['degree_center_dest'] = test_a['dest'].map(ref_dict2)

test = pd.concat([test_d,test_a], sort = False)

# 항공흐름 혼잡도 변수

In [ ]:
# 항공흐름 혼잡도 / 출도착 구분 
# 15분 전 항공흐름

# 출도착 구분

test['origin_dest'] = test['origin']+','+test['dest']

test_d = test[test['is_arrive']=='D']
test_a = test[test['is_arrive']=='A']

test_d = test_d.sort_values(['date','origin_dest','sched_time'])
test_a = test_a.sort_values(['date','origin_dest','sched_time'])

test_d.index = list(range(len(test_d)))

base_time = datetime.timedelta(minutes=15)
air_congestion_d = []

for i in range(len(test_d)):
    
    airway = test_d['origin_dest'][i]
    sched_time = test_d['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time 
    air_congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(test_d)-30) else len(test_d) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airway == test_d['origin_dest'][j]) and (lower_time <= test_d['sched_datetime'][j]) and (test_d['sched_datetime'][j] <= upper_time):
            air_congestion_cnt +=1
    air_congestion_d.append(air_congestion_cnt)
    
test_d['air_congestion_15'] = air_congestion_d


test_a.index = list(range(len(test_a)))

base_time = datetime.timedelta(minutes=15)
air_congestion_a = []

for i in range(len(test_a)):
    
    airway = test_a['origin_dest'][i]
    sched_time = test_a['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time 
    air_congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(test_a)-30) else len(test_a) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airway == test_a['origin_dest'][j]) and (lower_time <= test_a['sched_datetime'][j]) and (test_a['sched_datetime'][j] <= upper_time):
            air_congestion_cnt +=1
    air_congestion_a.append(air_congestion_cnt)
    
test_a['air_congestion_15'] = air_congestion_a

test = pd.concat([test_d, test_a], sort= False)

# 항공흐름 혼잡도 / 출도착 구분 
# 30분 전 항공흐름

# 출도착 구분

test_d = test[test['is_arrive']=='D']
test_a = test[test['is_arrive']=='A']

test_d = test_d.sort_values(['date','origin_dest','sched_time'])
test_a = test_a.sort_values(['date','origin_dest','sched_time'])

test_d.index = list(range(len(test_d)))

base_time = datetime.timedelta(minutes=30)
air_congestion_d = []

for i in range(len(test_d)):
    
    airway = test_d['origin_dest'][i]
    sched_time = test_d['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time 
    air_congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(test_d)-30) else len(test_d) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airway == test_d['origin_dest'][j]) and (lower_time <= test_d['sched_datetime'][j]) and (test_d['sched_datetime'][j] <= upper_time):
            air_congestion_cnt +=1
    air_congestion_d.append(air_congestion_cnt)
    
test_d['air_congestion_30'] = air_congestion_d


test_a.index = list(range(len(test_a)))

base_time = datetime.timedelta(minutes=30)
air_congestion_a = []

for i in range(len(test_a)):
    
    airway = test_a['origin_dest'][i]
    sched_time = test_a['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time 
    air_congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(test_a)-30) else len(test_a) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airway == test_a['origin_dest'][j]) and (lower_time <= test_a['sched_datetime'][j]) and (test_a['sched_datetime'][j] <= upper_time):
            air_congestion_cnt +=1
    air_congestion_a.append(air_congestion_cnt)
    
test_a['air_congestion_30'] = air_congestion_a

test = pd.concat([test_d, test_a], sort= False)

# 항공사의 일별 연결정도 변수

In [ ]:
# 항공사의 일별 연결정도 변수

test_d = test[test['is_arrive']=='D']
test_a = test[test['is_arrive']=='A']

# 출도착 일별 평균 비행기 개수

dd = pd.read_csv('bigcontest/test_ref/dd.csv', engine = 'python', encoding = 'cp949')
aa = pd.read_csv('bigcontest/test_ref/aa.csv', engine = 'python', encoding = 'cp949')

x = test_d.pivot_table(index = ['date','airline'],aggfunc = 'count', values = 'is_arrive')
x = x.reset_index(['date','airline'])
x.columns = ['date','airline','count']

y = test_a.pivot_table(index = ['date','airline'],aggfunc = 'count', values = 'is_arrive')
y = y.reset_index(['date','airline'])
y.columns = ['date','airline','count']

link_count = []
for i in range(len(x)):
    a = x['count'][i]
    b = float(dd[dd['airline']==x['airline'][i]]['count'].values)
    cnt = a/b
    link_count.append(cnt)
x['link_count'] = link_count

link_count = []
for i in range(len(y)):
    a = y['count'][i]
    b = float(aa[aa['airline']==y['airline'][i]]['count'].values)
    cnt = a/b
    link_count.append(cnt)
y['link_count'] = link_count

x = x.drop(columns = 'count')
y = y.drop(columns = 'count')

test_d = pd.merge(test_d,x, on = ['date','airline'])
test_a = pd.merge(test_a, y, on = ['date','airline'])

test = pd.concat([test_d, test_a], sort = False)

# 항공사별 time sequence 변수

In [ ]:
# 항공사별 sequence 생성

test_d = test[test['is_arrive']=='D']
test_a = test[test['is_arrive']=='A']

test_d = test_d.sort_values(['date','airline','sched_time'])
test_a = test_a.sort_values(['date','airline','sched_time'])

test_d.index = list(range(len(test_d)))
test_a.index = list(range(len(test_a)))

# 출발

air_link = []
x = 0

for i in range(len(test_d)):
    if i == 0:
        air_link.append(0)
    
    elif test_d['sched_datetime'][i] > test_d['sched_datetime'][i-1]:
        if test_d['day'][i] == test_d['day'][i-1]:
            x += 1
            air_link.append(x)
        elif test_d['day'][i] > test_d['day'][i-1]:
            x = 0
            air_link.append(x)
        elif test_d['day'][i] < test_d['day'][i-1]:
            x = 0
            air_link.append(x)
        
    elif test_d['sched_datetime'][i] == test_d['sched_datetime'][i-1]:
        air_link.append(x)
        
    elif test_d['sched_datetime'][i] < test_d['sched_datetime'][i-1]:
        x = 0
        air_link.append(x)

test_d['air_link'] = air_link

# 도착

air_link = []
x = 0

for i in range(len(test_a)):
    if i == 0:
        air_link.append(0)
    
    elif test_a['sched_datetime'][i] > test_a['sched_datetime'][i-1]:
        if test_a['day'][i] == test_a['day'][i-1]:
            x += 1
            air_link.append(x)
        elif test_a['day'][i] > test_a['day'][i-1]:
            x = 0
            air_link.append(x)
        elif test_a['day'][i] < test_a['day'][i-1]:
            x = 0
            air_link.append(x)
        
    elif test_a['sched_datetime'][i] == test_a['sched_datetime'][i-1]:
        air_link.append(x)
        
    elif test_a['sched_datetime'][i] < test_a['sched_datetime'][i-1]:
        x = 0
        air_link.append(x)
        

test_a['air_link'] = air_link


# 공항별 항공사별 sequence 생성

test_d = test_d.sort_values(['date','origin','airline','sched_time'])
test_a = test_a.sort_values(['date','dest','airline','sched_time'])

test_d.index = list(range(len(test_d)))
test_a.index = list(range(len(test_a)))

air_place_link = []
x = 0

for i in range(len(test_d)):
    if i == 0:
        air_place_link.append(0)
    
    elif (test_d['origin'][i] == test_d['origin'][i-1]) and (test_d['date'][i] == test_d['date'][i-1]) and (test_d['airline'][i] == test_d['airline'][i-1]):
        if test_d['sched_datetime'][i] > test_d['sched_datetime'][i-1]:
            x += 1
            air_place_link.append(x)
        elif test_d['sched_datetime'][i] == test_d['sched_datetime'][i-1]:
            air_place_link.append(x)
        elif test_d['sched_datetime'][i] < test_d['sched_datetime'][i-1]:
            x = 0
            air__placelink.append(x)
        
    else :
        x = 0
        air_place_link.append(x)

test_d['air_place_link'] = air_place_link

air_place_link = []
x = 0

for i in range(len(test_a)):
    if i == 0:
        air_place_link.append(0)
    
    elif (test_a['dest'][i] == test_a['dest'][i-1]) and (test_a['date'][i] == test_a['date'][i-1]) and (test_a['airline'][i] == test_a['airline'][i-1]):
        if test_a['sched_datetime'][i] > test_a['sched_datetime'][i-1]:
            x += 1
            air_place_link.append(x)
        elif test_a['sched_datetime'][i] == test_a['sched_datetime'][i-1]:
            air_place_link.append(x)
        elif test_a['sched_datetime'][i] < test_a['sched_datetime'][i-1]:
            x = 0
            air__placelink.append(x)
        
    else :
        x = 0
        air_place_link.append(x)

test_a['air_place_link'] = air_place_link

test = pd.concat([test_d, test_a], sort = False)

# 거리, 거리중심성 및 기타 공항 정보 변수

In [ ]:
# 거리 매핑 및 거리중심성 

distance = pd.read_csv('bigcontest/test_ref/airport_distance.csv', engine = 'python', encoding = 'cp949')
distance['origin_dest'] = distance['origin']+','+distance['dest']

ref_dict = {}

for i in range(len(distance)):
    ref_dict[distance['origin_dest'][i]] = distance['distance_km'][i]

test['distance_km'] = test['origin_dest'].map(ref_dict)

# 거리중심성 테입블 파일

center = pd.read_csv('bigcontest/test_ref/center.csv', encoding = 'cp949', engine = 'python')

# 매핑

ref_dict = {}

for i in range(len(center)):
    ref_dict[center['origin'][i]] = center['distance_center'][i]

test['distance_center'] = test['origin'].map(ref_dict)

# 거리중심성 분석
# 어떤 테이블이 있을건데 이것은 출발 - 도착 - 거리를 보여주는 데이터프레임 = df_distance 

origin_unique = (test['origin']).unique()

def get_distance_centrality(origin):
    N = 15
    df_distance = distance[distance['origin']==origin]
    df_distance.index = list(range(len(df_distance)))
    distance_sum = 0
    for i in range(len(df_distance)):
        distance_sum += df_distance['distance_km'][i]
            
    distance_sum_inverse = (distance_sum)**(-1)
    distance_center = (N-1)*distance_sum_inverse
    return distance_center 

# 중심성 분석 

origin_unique = (test['origin']).unique()
center_distance = pd.DataFrame(data=origin_unique,columns=['origin'])
D_list = []

for i in origin_unique:
    D = get_distance_centrality(i)
    D_list.append(D)

center_distance['distance_center'] = D_list

ref_dict = {}

for i in range(len(center_distance)):
    ref_dict[center_distance['origin'][i]] = center_distance['distance_center'][i]

test['distance_center_weight'] = test['origin'].map(ref_dict)

# 공항정보 매핑

info = pd.read_csv('bigcontest/test_ref/공항활주로_부지면적.csv', engine='python',encoding='cp949')
test = pd.merge(test, info, on=['origin','year'])

# 일별 비행기 운항횟수 변수

In [ ]:
# 하루당 해당 비행기 운행횟수

test_d = test[test['is_arrive']=='D']
test_a = test[test['is_arrive']=='A']

d_table = test_d.pivot_table(index=['date','tailnum'],aggfunc='count',values='is_arrive')
d_table = d_table.reset_index(['date','tailnum'])
d_table.columns = ['date','tailnum','cnt_per_day']

a_table = test_a.pivot_table(index=['date','tailnum'],aggfunc='count',values='is_arrive')
a_table = a_table.reset_index(['date','tailnum'])
a_table.columns = ['date','tailnum','cnt_per_day']

test_d2 = pd.merge(test_d, d_table, on = ['date','tailnum'])
test_a2 = pd.merge(test_a, a_table, on = ['date','tailnum'])

test = pd.concat([test_d2,test_a2], sort = False)

# 기타 시간 및 요일 변수 추가

In [ ]:
test['is_flight_T'] = np.where(test['flight'].str.endswith("T"), 1, 0)
test['is_flight_M'] = np.where(test['flight'].str.endswith("M"), 1, 0)
test['is_flight_F'] = np.where(test['flight'].str.endswith("F"), 1, 0)

import datetime as dt

test['sched_datetime'] = pd.to_datetime(test['sched_datetime'])

# 월
test['month_sin'] = np.sin(2*test['sched_datetime'].dt.month*np.pi / 12 )
test['month_cos'] = np.cos(2*test['sched_datetime'].dt.month*np.pi / 12 )

# 요일
test['sched_datetime'] = pd.to_datetime(test['sched_datetime'])
test['is_weekend'] = np.where((test['sched_datetime'].dt.weekday == 6) | (test['sched_datetime'].dt.weekday == 5), True, False)

# 시간
test['time_sin'] = 60 * np.sin(2*np.pi*test['sched_datetime'].dt.hour / 24) + \
                        np.sin(2*test['sched_datetime'].dt.minute*np.pi / 60)
test['time_cos'] = 60 * np.cos(2*test['sched_datetime'].dt.hour*np.pi / 24) + \
                        np.cos(2*np.pi*test['sched_datetime'].dt.minute / 60)




# 해발 고도
df_altitude = pd.DataFrame(pd.Series({
    '김포': 18,
    '김해': 4,
    '제주': 36,
    '대구': 35.4,
    '울산': 12.9,
    '청주': 58,
    '양양': 73.5,
    '무안': 23.7,
    '광주': 12.8,
    '여수': 20,
    '사천': 7.6,
    '포항': 21.3,
    '군산': 8.8,
    '원주': 100,
    '인천': 7
})).reset_index().rename(columns = {'index': 'airport', 0: 'alt'})

test = pd.merge(test, df_altitude, how = "left", left_on = 'origin', right_on = 'airport', suffixes = ('_origin', '_dest'))
test = pd.merge(test, df_altitude, how = "left", left_on = 'dest', right_on = 'airport', suffixes = ('_origin', '_dest'))
test.drop(['airport_origin', 'airport_dest'], axis = 1, inplace = True)


# is_변수 True False로 변환
test["is_arrive"] = np.where(test["is_arrive"] == "A", True, False)


# 황금연휴 추가

test['is_goldentime'] = test['date'].isin(["2017-01-26", "2017-01-27", "2017-01-28", "2017-01-29", "2017-01-30", "2017-01-31", "2017-05-05", "2017-05-06", "2017-05-07", "2017-08-15",
 "2017-09-29", "2017-09-30", "2017-10-01", "2017-10-02", "2017-10-03", "2017-10-04", "2017-10-05", "2017-10-06", "2017-10-07", "2017-10-08", "2017-10-09", "2017-10-10", 
                                                   "2017-12-25", "2018-01-01", "2018-02-14", "2018-02-15", "2018-02-16", "2018-02-17", "2018-02-18", "2018-02-19", "2018-03-01",
 "2018-05-05", "2018-05-07", "2018-05-22",
 "2018-06-06", "2018-06-13", "2018-08-15", "2018-09-21", "2018-09-22", "2018-09-23", "2018-09-24", "2018-09-25", "2018-09-26", "2018-09-27", 
 "2018-10-03", "2018-10-09", "2018-12-25", 
 "2019-01-01", "2019-02-01", "2019-02-02", "2019-02-03", "2019-02-04", "2019-02-05", "2019-02-06", "2019-02-07",
 "2019-03-01", "2019-05-04", "2019-05-05", "2019-05-06"])

# 전 비행기와의 시간 차이 변수

In [ ]:
# 연결 비행기에 있어서 전 비행기와의 시간차이 및 거리를 고려한 시간차이 


test_d = test[test['is_arrive']==False]
test_a = test[test['is_arrive']==True]

# 출발인경우

test_d_sorted = test_d.sort_values(['date','tailnum','sched_time'])
test_d_sorted.index = list(range(len(test_d_sorted)))

time_diff = []
time_diff_distance = []

for i in range(len(test_d_sorted)):
    if test_d_sorted['link'][i] == 0 :
        time_diff.append(0)
        time_diff_distance.append(0)
    
    elif test_d_sorted['tailnum'][i] != test_d_sorted['tailnum'][i-1]:
        time_diff.append(0)
        time_diff_distance.append(0)

    else :
        time_diff_sec = (test_d_sorted['sched_datetime'][i] - test_d_sorted['sched_datetime'][i-1]).seconds
        time_diff_min = int(time_diff_sec/60)
        time_diff.append(time_diff_min)
        
        time_diff_dis = time_diff_min/test_d_sorted['distance_km'][i-1]
        time_diff_distance.append(time_diff_dis)
        
test_d_sorted['time_diff'] = time_diff
test_d_sorted['time_diff_distance'] = time_diff_distance

# 도착인경우

test_a_sorted = test_a.sort_values(['date','tailnum','sched_time'])
test_a_sorted.index = list(range(len(test_a_sorted)))

time_diff = []
time_diff_distance = []

for i in range(len(test_a_sorted)):
    if test_a_sorted['link'][i] == 0 :
        time_diff.append(0)
        time_diff_distance.append(0)
    
    elif test_a_sorted['tailnum'][i] != test_a_sorted['tailnum'][i-1]:
        time_diff.append(0)
        time_diff_distance.append(0)
           
    else :
        time_diff_sec = (test_a_sorted['sched_datetime'][i] - test_a_sorted['sched_datetime'][i-1]).seconds
        time_diff_min = int(time_diff_sec/60)
        time_diff.append(time_diff_min)
        
        time_diff_dis = time_diff_min/test_a_sorted['distance_km'][i-1]
        time_diff_distance.append(time_diff_dis)
        
test_a_sorted['time_diff'] = time_diff
test_a_sorted['time_diff_distance'] = time_diff_distance

test = pd.concat([test_d_sorted, test_a_sorted], sort = False)

# 항공편 지연비율 변수

In [ ]:
test_d = test[test['is_arrive']==False]
test_a = test[test['is_arrive']==True]

d = pd.read_csv('bigcontest/test_ref/flight_dly_d.csv', engine = 'python', encoding = 'cp949')
a = pd.read_csv('bigcontest/test_ref/flight_dly_a.csv', engine = 'python', encoding = 'cp949')


test_d = pd.merge(test_d, d, on = 'flight', how = 'left')
test_d['dly_por'].fillna(d['dly_por'].mean(), inplace = True)
test_a = pd.merge(test_a, a, on = 'flight', how = 'left')
test_a['dly_por'].fillna(a['dly_por'].mean(), inplace = True)
test = pd.concat([test_d, test_a], sort=False)

test.to_csv('bigcontest/testset2.csv', index=False, encoding = 'cp949')

# 트레이닝 셋에 기상 매핑

-  학습을 목적으로 17년~19년 6월 데이터 셋에 기상을 한다.

In [ ]:
weather_for_train=pd.read_csv('bigcontest/airport_weather.csv',encoding='cp949')
weather_for_train.time_slot=pd.to_datetime(weather_for_train.time_slot)

#### 기상변수 이름 테스트 셋과 동일하게 변환

In [ ]:
weather_for_train.rename({'temp':'pred_temp','dew':'pred_dew'},axis=1,inplace=True)

#### 습도 변수 생성

In [ ]:
weather_for_train['mist']=weather_for_train.pred_temp-weather_for_train.pred_dew

#### 파생변수 - 전 시간대와 온도차 변수 생성

In [ ]:
#이슬점 차

def create_diff_dew(df):
    dew = list(df.pred_dew)
    # 첫 번째 행은 비교할 전 시간대가 없으므로 기온차 값을 0으로 만들기 위해 자기 자신값과 비교
    dew.insert(0, dew[0])
    # 맨 뒤 값 제거
    dew.pop() 
    df['before_dew'] = dew
    # 현재 시간대 기온 - 이전 시간대 기온
    diff_list = list(df.pred_dew - df.before_dew)
    # 공항별 sorting했기 때문에 각 공항의 첫 번째 행을 모두 0으로 만들어주기 위함.
    firstday_list = []
    for arp in list(set(df.mapping_arp)):
        firstday_list.append(df[df.mapping_arp == arp].index[0])
    for i in firstday_list:
        diff_list[i] = 0
    df['diff_dew'] = diff_list
    df.drop('before_dew', axis=1, inplace=True)
    
    return df

# 기온 차
def create_diff_temp(df):
    temp = list(df.pred_temp)
    # 첫 번째 행은 비교할 전 시간대가 없으므로 기온차 값을 0으로 만들기 위해 자기 자신값과 비교
    temp.insert(0, temp[0])
    # 맨 뒤 값 제거
    temp.pop() 
    df['before_temp'] = temp
    # 현재 시간대 기온 - 이전 시간대 기온
    diff_list = list(df.pred_temp - df.before_temp)
    # 공항별 sorting했기 때문에 각 공항의 첫 번째 행을 모두 0으로 만들어주기 위함.
    firstday_list = []
    for arp in list(set(df.mapping_arp)):
        firstday_list.append(df[df.mapping_arp == arp].index[0])
    for i in firstday_list:
        diff_list[i] = 0
    df['diff_temp'] = diff_list
    df.drop('before_temp', axis=1, inplace=True)
    
    return df

In [ ]:
weather_for_train.sort_values(['mapping_arp','time_slot'], inplace=True)

In [ ]:

weather_for_train =  create_diff_temp(weather_for_train)
weather_for_train = create_diff_dew(weather_for_train)

In [ ]:
train_add_weather=pd.read_csv('bigcontest/df_afsnt_total_ver10.csv',encoding='cp949')

In [ ]:
#train_add_weather.drop(['pred_temp','diff_temp'],axis=1,inplace=True)

In [ ]:
#train_add_weather.sched_datetime=train_add_weather.sched_datetime+':00'

#### 매핑용 time_slot 생성

In [ ]:
def create_time_slot(date):
    m = date.split(' ')[1].split(':')[1]
    # 30분 미만일 때 00분으로
    if int(m) < 30:
        result = date[:13]+':00:00'
    # 30분 이상일 때 다음 시간대로 넘어감.
    else:
        # ex) 7시->07시
        if len(str(int(date[11:13]))) == 1:
            # 9시->010시 방지.
            if date[11:13]=='09':
                result = date[:11]+str(int(date[11:13])+1)+':00:00'
            else:
                result = date[:11]+'0'+str(int(date[11:13])+1)+':00:00'
        # ex) 23시 35분->23시 00분,
        elif date[11:13]=='23':
            result = date[:13]+':00:00'
        else: 
            result = date[:11]+str(int(date[11:13])+1)+':00:00'
    return result#### 매핑용 time_slot 생성

#### 매핑용 기상 공항 생성

In [ ]:
train_add_weather['time_slot'] = train_add_weather.sched_datetime.apply(create_time_slot)
train_add_weather['mapping_arp'] = train_add_weather.origin.map({'포항': '울산', '사천': '여수',
                                           '군산': '무안', '원주': '김포',
                                           '김해': '울산', '대구': '울산',
                                           '청주': '김포', '광주': '무안',
                                           '김포': '김포', '무안': '무안',
                                           '양양': '양양', '여수': '여수',
                                           '울산': '울산', '인천': '인천',
                                           '제주': '제주'})

### 실제 기상 매핑 완료

In [ ]:
weather_for_train.time_slot = weather_for_train.time_slot.apply(str)
train_add_weather = pd.merge(train_add_weather, weather_for_train, on=['mapping_arp','time_slot'], how='left')
train_add_weather.drop(['mapping_arp','time_slot'], axis=1, inplace=True)

In [ ]:
train_add_weather.to_csv('df_afsnt_total_ver10+weather.csv',index=False,encoding='CP949')

# 테스트 셋 매핑용 기상 예측

### 19년 9월 기상 예측

-  19년 9월 기상 예측을 위해 15년도~19년도의 8월, 9월, 10월 시간별/공항별 기상을 가져온다.

In [ ]:
weather=pd.read_csv('bigcontest/airport_weather.csv',encoding='cp949')
encode_case=pd.get_dummies(weather.mapping_arp)
weather=pd.concat([weather,encode_case],axis=1)
# 범주형 변수인 공항을 원핫 인코딩으로 변환

weather.head()

- 시간을 연,월,일,시간대로 나눠주어서 X feature로 사용

In [ ]:
# time 을 시간 단위 별로 구분해주는 함수
def date_segmentation(data):
    ls_date=list(data['time_slot'])

    월=[]
    시간=[]
    시간_2=[]

    일시=[]
    일=[]
    연=[]

#for i in range(len(ls_date)):
#    연.append(str(ls_date[i]).split('-')[0])

    for i in range(len(ls_date)):
        연.append(str(ls_date[i]).split('-')[0])
        
    for i in range(len(ls_date)):
        월.append(str(ls_date[i]).split('-')[1])
    for i in range(len(ls_date)):
        시간.append(str(ls_date[i]).split(' ')[1])

    for i in range(len(ls_date)):
        시간_2.append(str(시간[i]).split(':')[0])

    
    for i in range(len(ls_date)):
        일시.append(str(ls_date[i]).split(' ')[0])

    for i in range(len(ls_date)):    
        일.append(str(일시[i]).split('-')[2])


    data['연']=연
    data['월']=월
    data['일']=일
    data['시각']=시간_2
    return data

In [ ]:

weather=date_segmentation(weather)
weather=weather.sort_values(by=['time_slot','mapping_arp'])
weather.reset_index(drop=True,inplace=True)

weather.set_index('time_slot',inplace=True)

weather_sep=weather[weather['월'].isin(['08','09'])]
#test 셋인 8월의 날씨를 학습하는 weather_june과
#실제 공모전 validation 답안 제출을 위한 학습하는 데이터셋 weather_sep을 구분하여 생성

## 9월 예측 구간 온도 예측 By 8,9월 기상

In [ ]:

test=weather_sep['2019-09-01':'2019-09-09'] #검증 구간

train=weather_sep['2015-08-01':'2019-09-09'] # 학습 구간 
# 실제 공모전 및 제출 기간을 고려하여 validation의 구간 결정


X_train = train.drop(['mapping_arp','temp','dew'],axis=1).values[:]
y_train = train['temp'].values[:]
    
X_test = np.array(test.drop(['mapping_arp','temp','dew'],axis=1).values[:])
y_test = np.array(test['temp'].values[:])

### LightGBM 모델을 통해 학습

In [ ]:
train_ds = lgb.Dataset(X_train, label = y_train) 
test_ds = lgb.Dataset(X_test, label = y_test) 
params = {'learning_rate': '0.254', 
          'max_depth': 3, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'rmse', 
          'is_training_metric': True, 
          'num_leaves': 30, 
          'feature_fraction': '0.01',
          'bagging_fraction': '0.036',
          "bagging_seed" : 5,
          'bagging_freq': 20, 
          'seed':3}

model_temp = lgb.train(params, train_ds, 1000, test_ds,\
                  verbose_eval=100, early_stopping_rounds=200)

predict_train = model_temp.predict(X_train)
predict_test = model_temp.predict(X_test)

mse = mean_squared_error(y_test, predict_test)

print('rmse: ', np.sqrt(mse))


#생성된 모델을 통해 6월 15일~ 6월 30일 구간의 기상 값을 예측


## 9월 예측 구간 이슬점 예측 By 8,9월 기상

In [ ]:

test=weather_sep['2019-09-01':'2019-09-09'] #검증 구간

train=weather_sep['2015-08-01':'2019-09-09'] # 학습 구간 
# 실제 공모전 및 제출 기간을 고려하여 validation의 구간 결정


X_train = train.drop(['mapping_arp','dew','temp'],axis=1).values[:]
y_train = train['dew'].values[:]
    
X_test = np.array(test.drop(['mapping_arp','dew','temp'],axis=1).values[:])
y_test = np.array(test['dew'].values[:])

### LightGBM 모델을 통해 학습

In [ ]:
train_ds = lgb.Dataset(X_train, label = y_train) 
test_ds = lgb.Dataset(X_test, label = y_test) 
params = {'learning_rate': '0.254', 
          'max_depth': 3, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'rmse', 
          'is_training_metric': True, 
          'num_leaves': 30, 
          'feature_fraction': '0.01',
          'bagging_fraction': '0.036',
          "bagging_seed" : 5,
          'bagging_freq': 20, 
          'seed':3}

model_dew = lgb.train(params, train_ds, 1000, test_ds,\
                  verbose_eval=100, early_stopping_rounds=200)

predict_train = model_dew.predict(X_train)
predict_test = model_dew.predict(X_test)

mse = mean_squared_error(y_test, predict_test)

print('rmse: ', np.sqrt(mse))


#생성된 모델을 통해 6월 15일~ 6월 30일 구간의 기상 값을 예측


## 실제 기상 예측

 #### AFSNT_DLY 예측 구간용 X_feature 구축

In [ ]:
# 공모전 평가 기간인 9월 16일~ 9월 30일 용 예측 기상 추출 후 매핑
from datetime import timedelta
test_period = weather['2018-09-15':'2018-09-30']
test_period.reset_index(inplace=True)
test_period['time_slot']=pd.to_datetime(test_period.time_slot)
test_period.time_slot =test_period.time_slot  + timedelta(days = 366)

In [ ]:
test_period['연']=2019
X_test = np.array(test_period[['김포', '무안', '양양', '여수', '울산', '인천', '제주', '연', '월', '일',
       '시각']].values[:])
pred_temp = model_temp.predict(X_test)
pred_dew = model_dew.predict(X_test)
test_period.drop('temp',axis=1,inplace=True)
test_period['pred_temp']=pred_temp
test_period.drop('dew',axis=1,inplace=True)
test_period['pred_dew']=pred_dew
test_period.head()

#9월 16일부터 15일 간 00시부터 23시까지의 예측된 온도 값 생성

In [ ]:
test_period['mist']=pred_temp-pred_dew

#### 파생변수 - 전 시간대와 온도차 변수 생성

In [ ]:
#이슬점 차

def create_diff_dew(df):
    dew = list(df.pred_dew)
    # 첫 번째 행은 비교할 전 시간대가 없으므로 기온차 값을 0으로 만들기 위해 자기 자신값과 비교
    dew.insert(0, dew[0])
    # 맨 뒤 값 제거
    dew.pop() 
    df['before_dew'] = dew
    # 현재 시간대 기온 - 이전 시간대 기온
    diff_list = list(df.pred_dew - df.before_dew)
    # 공항별 sorting했기 때문에 각 공항의 첫 번째 행을 모두 0으로 만들어주기 위함.
    firstday_list = []
    for arp in list(set(df.mapping_arp)):
        firstday_list.append(df[df.mapping_arp == arp].index[0])
    for i in firstday_list:
        diff_list[i] = 0
    df['diff_dew'] = diff_list
    df.drop('before_dew', axis=1, inplace=True)
    
    return df

# 기온 차
def create_diff_temp(df):
    temp = list(df.pred_temp)
    # 첫 번째 행은 비교할 전 시간대가 없으므로 기온차 값을 0으로 만들기 위해 자기 자신값과 비교
    temp.insert(0, temp[0])
    # 맨 뒤 값 제거
    temp.pop() 
    df['before_temp'] = temp
    # 현재 시간대 기온 - 이전 시간대 기온
    diff_list = list(df.pred_temp - df.before_temp)
    # 공항별 sorting했기 때문에 각 공항의 첫 번째 행을 모두 0으로 만들어주기 위함.
    firstday_list = []
    for arp in list(set(df.mapping_arp)):
        firstday_list.append(df[df.mapping_arp == arp].index[0])
    for i in firstday_list:
        diff_list[i] = 0
    df['diff_temp'] = diff_list
    df.drop('before_temp', axis=1, inplace=True)
    
    return df

In [ ]:
test_period.sort_values(['mapping_arp','time_slot'], inplace=True)
test_period =  create_diff_temp(test_period)
test_period = create_diff_dew(test_period)

In [ ]:
test_period.to_csv('bigcontest/test_period_pred_weather.csv',index=False,encoding='cp949')

In [ ]:
df_pred = pd.read_csv('bigcontest/test_period_pred_weather.csv', encoding='cp949')
df_pred.reset_index(inplace=True)
df_pred.drop('index', axis=1, inplace=True)
df_pred.drop(['김포','무안','양양','여수','울산','인천','제주','연','월','일','시각'], axis=1, inplace=True)

In [ ]:
df_dly=pd.read_csv('bigcontest/testset2.csv',encoding='cp949')

#### 매핑용 time_slot 생성

In [ ]:
def create_time_slot(date):
    m = date.split(' ')[1].split(':')[1]
    # 30분 미만일 때 00분으로
    if int(m) < 30:
        result = date[:13]+':00:00'
    # 30분 이상일 때 다음 시간대로 넘어감.
    else:
        # ex) 7시->07시
        if len(str(int(date[11:13]))) == 1:
            # 9시->010시 방지.
            if date[11:13]=='09':
                result = date[:11]+str(int(date[11:13])+1)+':00:00'
            else:
                result = date[:11]+'0'+str(int(date[11:13])+1)+':00:00'
        # ex) 23시 35분->23시 00분,
        elif date[11:13]=='23':
            result = date[:13]+':00:00'
        else: 
            result = date[:11]+str(int(date[11:13])+1)+':00:00'
    return result



#### 매핑용 기상 공항 생성

In [ ]:
df_dly['time_slot'] = df_dly.sched_datetime.apply(create_time_slot)
df_dly['mapping_arp'] = df_dly.origin.map({'포항': '울산', '사천': '여수',
                                           '군산': '무안', '원주': '김포',
                                           '김해': '울산', '대구': '울산',
                                           '청주': '김포', '광주': '무안',
                                           '김포': '김포', '무안': '무안',
                                           '양양': '양양', '여수': '여수',
                                           '울산': '울산', '인천': '인천',
                                           '제주': '제주'})

In [ ]:
df_dly = pd.merge(df_dly, df_pred, on=['mapping_arp','time_slot'], how='left')
df_dly.drop(['mapping_arp','time_slot'], axis=1, inplace=True)

In [ ]:
df_dly.columns

In [ ]:
df_dly.to_csv('bigcontest/all_merged_test_set.csv',index=False,encoding='CP949')

In [ ]:
df_afsnt = pd.read_csv('bigcontest/df_afsnt_total_ver10+weather.csv',engine = 'python', encoding = 'cp949')
test = pd.read_csv('bigcontest/all_merged_test_set.csv', engine = 'python', encoding = 'cp949')
C10date = ['2017-12-18 00:00:00',
 '2019-02-15 00:00:00',
 '2018-11-24 00:00:00',
 '2018-02-04 00:00:00',
 '2017-01-20 00:00:00',
 '2018-01-13 00:00:00',
 '2018-02-23 00:00:00',
 '2018-02-06 00:00:00',
 '2018-01-24 00:00:00',
 '2018-02-05 00:00:00',
 '2018-02-08 00:00:00',
 '2017-02-10 00:00:00',
 '2018-01-26 00:00:00',
 '2017-02-22 00:00:00',
 '2019-01-14 00:00:00',
 '2017-12-07 00:00:00']
df_afsnt= df_afsnt[~((df_afsnt.date.isin(C10date) ==True) & (df_afsnt.cause_delay =='C10'))]
df_afsnt = df_afsnt.drop(columns = ['origin_dest','pred_dew'])
test = test.drop(columns = ['origin_dest','pred_dew'])
# 시간정렬
df_afsnt = df_afsnt.sort_values(['year', 'month', 'day'], ascending=[True, True, True])
df_afsnt = df_afsnt.reset_index()
df_afsnt.drop('index', axis = 1, inplace = True)
# 시간정렬
testt = test.sort_values(['year', 'month', 'day'], ascending=[True, True, True])
test = test.reset_index()
test.drop('index', axis = 1, inplace = True)

print(__doc__)
import lightgbm as lgb
from datetime import datetime 

import numpy as np
from scipy import interp
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc
'''
Trainining set과 Validation set의 차이가 77일이고 Validation set의 길이가 15일인 Time Series Cross Validation을 수행합니다.

각 Fold의 전체 Validation Set으로 예측한 뒤, Validation Set의 결항 데이터를 제거하여 평가합니다.
'''
# CV Class
class BigconTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
        
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y = None, groups = None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)
        
        # fold마다 margin을 달리 잡는다.
        margin = 1050*77
        start = 0
        stop = 0 
        # 17년 6월, 17년 9월, 18년 6월, 18년 9월, 19년 6월을 val set으로 잡았다.
        # fold2와 fold4는 17년 9월15일, 18년 9월 27일이 val set의 시작이다.
        mid = [1050*30*6, 1050*30*9 - 1050*3, 1050*30*18, 1050*30*21 + 1050*20, 5 * k_fold_size - 1050*15]
        stop = [1050*30*6 + 1050*15, 1050*30*9 + 1050*15 - 1050*3, 1050*30*18 + 1050*15, 1050*30*21 + 1050*15 + 1050*20, 5 * k_fold_size]
        for i in range(self.n_splits):
            #stop = (i + 1) *  k_fold_size
            #mid = stop - 1050*15
            yield indices[start:mid[i] - margin], indices[mid[i]: stop[i]]
            
random_state = 8282
bts_cv = BigconTimeSeriesSplit(n_splits=5)

# 결항데이터 제거용
# y_cancel = pd.Series(np.where(df_afsnt_total['is_cancel'] == "Y", True, False))
# idx_cancel = y_cancel[y_cancel == True].index

df_afsnt_total = df_afsnt
df_afsnt_total = df_afsnt_total.drop(['month','day','flight','origin','dest','cause_cancel','tailnum','cause_delay','sched_time','real_time',
        'is_cancel', 'date','sched_datetime','real_datetime','delay', 'is_regular'], axis=1)

# 요일
encoded = pd.get_dummies(df_afsnt_total.wday)
df_afsnt_total = pd.concat([df_afsnt_total, encoded], axis=1)
df_afsnt_total.drop('wday', axis = 1, inplace = True)

# 시간범주화
encoded = pd.get_dummies(df_afsnt_total.time_discrete)
df_afsnt_total = pd.concat([df_afsnt_total, encoded], axis=1)
df_afsnt_total.drop('time_discrete', axis = 1, inplace = True)

# 항공사
encoded = pd.get_dummies(df_afsnt_total.airline)
df_afsnt_total = pd.concat([df_afsnt_total, encoded], axis=1)
df_afsnt_total.drop('airline', axis = 1, inplace = True)


# is_arrive
encoded = pd.get_dummies(df_afsnt_total.is_arrive)
df_afsnt_total = pd.concat([df_afsnt_total, encoded], axis=1)
df_afsnt_total.drop('is_arrive', axis = 1, inplace = True)


#origin_dest
#encoded = pd.get_dummies(df_afsnt_total.origin_dest)
#df_afsnt_total = pd.concat([df_afsnt_total, encoded], axis=1)
#df_afsnt_total.drop('origin_dest', axis = 1, inplace = True)

clf = lgb.LGBMClassifier(boosting_type= 'dart', colsample_by_tree= 0.6533437523326235, learning_rate= 0.04160534135601695, 
                              max_depth= 19, n_estimators= 2000, num_leaves= 35, reg_lambda= 0.5508858320081631, class_weight='balanced')




#clf = lgb.LGBMClassifier(max_depth=-1, learning_rate=0.05, objective='binary',
#                         random_state=8282, silent=True, metric='None', 
#                         n_estimators=500, class_weight='balanced',colsample_bytree = 0.9,
# min_child_samples = 68,
# num_leaves = 100,
# subsample = 0.9)

y = df_afsnt_total['is_delay'] 
X = df_afsnt_total.loc[:, df_afsnt_total.columns != 'is_delay']


# 요일
encoded = pd.get_dummies(test.wday)
test = pd.concat([test, encoded], axis=1)
test.drop('wday', axis = 1, inplace = True)


# 시간범주화
encoded = pd.get_dummies(test.time_discrete)
test = pd.concat([test, encoded], axis=1)
test.drop('time_discrete', axis = 1, inplace = True)

# 항공사
encoded = pd.get_dummies(test.airline)
test = pd.concat([test, encoded], axis=1)
test.drop('airline', axis = 1, inplace = True)


# is_arrive
encoded = pd.get_dummies(test.is_arrive)
test = pd.concat([test, encoded], axis=1)
test.drop('is_arrive', axis = 1, inplace = True)
df_test = test.iloc[:, test.columns.isin(X.columns)] 

print("모델을 훈련합니다.")
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

for fold_num, (train_idx, val_idx) in enumerate(bts_cv.split(X, y), 1):
        # Fold의 피팅 시작시간 
        start_time = datetime.now()
        
        X_train, y_train = X.iloc[train_idx, :], y.loc[train_idx]
        X_val, y_val = X.iloc[val_idx, :], y.loc[val_idx]
        
        # 훈련
        probas_ = pd.DataFrame({'idx_val': val_idx, 'prob': clf.fit(X_train, y_train).predict_proba(X_val)[:, 1]})               
        
#         # 종속변수의 결항데이터 제거 
#         val_idx = list(np.setdiff1d(val_idx, idx_cancel))
#         probas_ = probas_[probas_.isin(val_idx)['idx_val']]
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(y[val_idx], probas_.iloc[:, 1])
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        plt.plot(fpr, tpr, lw=1, alpha=0.3,
                 label='ROC fold %d (AUC = %0.4f)' % (fold_num, roc_auc))
        
        # Fold의 피팅 소요시간 
        time_elapsed = datetime.now() - start_time
        print('{0} Fold fitting time (hh:mm:ss.ms) {1}'.format(fold_num, time_elapsed))


plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.4f $\pm$ %0.4f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()